# 导入数据
借助 tf.data API，可以根据简单的可重用片段构建复杂的输入管道。

例如，

图片模型的管道可能会汇聚分布式文件系统中的文件中的数据、对每个图片应用随机扰动，并将随机选择的图片合并成用于训练的批次。
文本模型的管道可能包括从原始文本数据中提取符号、根据对照表将其转换为嵌入标识符，以及将不同长度的序列组合成批次数据。

使用 tf.data API 可以轻松处理大量数据、不同的数据格式以及复杂的转换。

tf.data API 在 TensorFlow 中引入了两个新的抽象类：

`tf.data.Dataset` 

表示一系列元素，其中每个元素包含一个或多个 Tensor 对象。

例如，在图像管道中，元素可能是单个训练样本，具有一对表示图像数据和标签的张量。

可以通过两种不同的方式来创建数据集：

- 创建来源（例如 Dataset.from_tensor_slices()），以通过一个或多个 tf.Tensor 对象构建数据集。

- 应用转换（例如 Dataset.batch()），以通过一个或多个 tf.data.Dataset 对象构建数据集。

`tf.data.Iterator` 
提供了从数据集中提取元素的主
要方法。Iterator.get_next() 返回的操作会在执行时生成 Dataset 的下一个元素，并且此操作通常充当输入管道代码和模型之间的接口。

最简单的迭代器是“单次迭代器”，它与特定的 Dataset 相关联，并对其进行一次迭代。

要实现更复杂的用途，可以通过 Iterator.initializer 操作使用不同的数据集重新初始化和参数化迭代器，这样一来，您就可以在同一个程序中对训练和验证数据进行多次迭代（举例而言）。

## 基本机制
这部分介绍创建不同种类的 Dataset 和 Iterator 对象的基础知识，以及如何从这些对象中提取数据。

### 数据定义
要启动输入管道，您必须定义来源。

例如，要通过内存中的某些张量构建 Dataset，可以使用 tf.data.Dataset.from_tensors() 或 tf.data.Dataset.from_tensor_slices()。

或者，如果输入数据以推荐的 TFRecord 格式存储在磁盘上，那么可以构建 tf.data.TFRecordDataset。

一旦有了 Dataset 对象，可以将其转换为新的 Dataset，方法是tf.data.Dataset 对象上的方法调用。

例如，可以应用单元素转换，例如 Dataset.map()（为每个元素应用一个函数），也可以应用多元素转换（例如 Dataset.batch()）。

### 数据消费
消耗 Dataset 中值的最常见方法是构建迭代器对象。
通过此对象，可以一次访问数据集中的一个元素（例如通过调用 Dataset.make_one_shot_iterator()）。

tf.data.Iterator 提供了两个操作：

#### Iterator.initializer
可以通过此操作（重新）初始化迭代器的状态；

#### Iterator.get_next()
此操作返回对应下一个元素的 tf.Tensor 对象。

根据使用情形，可以选择不同类型的迭代器，下面具体选项。

## 数据集结构
一个数据集包含多个元素，每个元素的结构都相同。

可以通过 Dataset.output_types 和 Dataset.output_shapes 属性检查数据集元素各个组件的类型和形状。

这些属性的嵌套结构映射到元素的结构，此元素可以是单个张量、张量元组，也可以是张量的嵌套元组。例如：

In [2]:
import tensorflow as tf
print(tf.random_uniform([4, 10]))
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4, 10]))
print(dataset1.output_types)  # ==> "tf.float32"
print(dataset1.output_shapes)  # ==> "(10,)"

Tensor("random_uniform:0", shape=(4, 10), dtype=float32)
<dtype: 'float32'>
(10,)


## 迭代器
如果 Dataset 是一个水池的话，那么它其中的数据就好比是水池中的水，Iterator 你可以把它当成是一根水管。

在 Tensorflow 的程序代码中，正是通过 Iterator 这根水管，才可以源源不断地从 Dataset 中取出数据。

但为了应付多变的环境，水管也需要变化，Iterator 也有许多种类。

tf.data API 目前支持下列迭代器，复杂程度逐渐增大：

- 单次（one-shot）
- 可初始化（initializable）, 接受参数，可以定制化的水管
- 可重新初始化(reinitializable)， 能够接受不同输入的
- 可馈送(feedable)

### 单次（one-shot）
创建单次迭代器，非常的简单，只需要调用 Dataset 对象相应的`make_one_shot_iterator()`方法。

这个方法会返回一个 Iterator 对象。

而调用 iterator 的 get_next() 就可以轻松地取出数据了。

In [4]:
sess=tf.InteractiveSession()

dataset = tf.data.Dataset.range(10)
dataset = dataset.batch(1)
# make_one_shot_iterator 一次访问Dataset中的一个元素
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

for i in range(10):
    value = sess.run(next_element)
    print(value)
    #assert i == value

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]


上面的代码非常简单，首先创建了一个包含 0 到 10 的数据集。然后，创建了一个单次迭代器。

通过循环调用 get_next() 方法就可以将数据取出。

**需要注意的是**，

通常用 try-catch 配合使用，当 Dataset 中的数据被读取完毕的时候，程序会抛出异常，获取这个异常就可以从容结束本次数据的迭代。

然后， iterator 就完成了它的历史使命。单次的迭代器，不支持动态的数据集，它比较单纯，它不支持参数化。

什么是参数化呢？你可以理解为单次的 Iterator需要 Dataset 在程序运行之前就确认自己的大小，

但我们都知道 Tensorflow 中有一种 feeding 机制，它允许我们在程序运行时再真正决定我们需要的数据，单次的 Iterator **不能满足这要的要求**。


### 可初始化（initializable）

需要先运行显式 iterator.initializer 操作，然后才能使用可初始化迭代器。

虽然有些不便，但它允许您使用一个或多个 tf.placeholder() 张量（可在初始化迭代器时传递参数）参数化数据集的定义。

继续以 Dataset.range() 为例：

In [5]:
max_value = tf.placeholder(tf.int64, shape=[])
dataset = tf.data.Dataset.range(max_value)
# make_initializable_iterator 初始化迭代器
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

# Initialize an iterator over a dataset with 10 elements.
sess.run(iterator.initializer, feed_dict={max_value: 10})
for i in range(10):
    value = sess.run(next_element)
    assert i == value

# Initialize the same iterator over a dataset with 100 elements.
sess.run(iterator.initializer, feed_dict={max_value: 100})
for i in range(100):
    value = sess.run(next_element)
    assert i == value

Instructions for updating:
Colocations handled automatically by placer.


跟单次 Iterator 的代码只有 2 处不同。

1、创建的方式不同，iterator.make_initialnizer()。

2、每次重新初始化的时候，都要调用sess.run(iterator.initializer)

你可以这样理解，Dataset 这个水池连续装了 2 次水，每次水量不一样，但可初始化的 Iterator 很好地处理了这件事情，但需要注意的是，这个时候 Iterator 还是面对同一个 Dataset。


### 可重新初始化(reinitializable)

有时候，需要一个 Iterator 从不同的 Dataset 对象中读取数值。Tensorflow 针对这种情况，提供了一个可以重新初始化的 Iterator，它的用法相对而言，比较复杂，但好在不是很难理解。

In [6]:
training_data = tf.data.Dataset.range(10)
validation_data = tf.data.Dataset.range(5)

# 核心代码
# Iterator 可以接多个水池里面的水，但是要求这水池里面的水是同样的品质。
# 也就是，多个 Dataset 中它们的元素数据类型和形状应该是一致的。
# 通过 from_structure() 统一规格，后面的 2 句代码可以看成是 2 个水龙头，它们决定了放哪个水池当中的水。
iterator = tf.data.Iterator.from_structure(training_data.output_types,
                                               training_data.output_shapes)

train_op = iterator.make_initializer(training_data)
validation_op = iterator.make_initializer(validation_data)
next_element = iterator.get_next()

# train 3 epochs
for _ in range(3):
    # initializae an iterator over training dataset
    sess.run(train_op)
    print('Training begin')
    for _ in range(3):
        print(sess.run(next_element))
    print('Training end')
    print('Validating begin')
    # initializae an iterator over validation dataset
    sess.run(validation_op)
    for _ in range(2):
        print(sess.run(next_element))
    print('Validating end')



Training begin
0
1
2
Training end
Validating begin
0
1
Validating end
Training begin
0
1
2
Training end
Validating begin
0
1
Validating end
Training begin
0
1
2
Training end
Validating begin
0
1
Validating end


每次 Iterator 切换时，数据都从头开始打印了。

如果，不想这种情况发生，需要使用可馈送(feedable)的iterator

### 可馈送(feedable)

Tensorflow中的feeding机制帮助我们解决了很多问题，它决定了很多东西可以在程序运行时，动态填充，这其中也包括了 Iterator。

也就是，不同的 Dataset 用不同的 Iterator，然后利用 feeding 机制，动态决定。

我们都知道，无论是在机器学习还是深度学习当中，训练集、验证集、测试集是大家绕不开的话题，但偏偏它们要分离开来，偏偏它们的数据类型又一致，所以，经常我们要写同样的重复的代码。

复用，是软件开发中一个重要的思想。

可馈送的 Iterator 一定程度上可以解决重复的代码，同时又将训练集和验证集的操作清晰得分离开来。


In [6]:
# Define training and validation datasets with the same structure.
training_dataset = tf.data.Dataset.range(100).map(
    lambda x: x + tf.random_uniform([], -10, 10, tf.int64))
validation_dataset = tf.data.Dataset.range(50)

# 是通过一个 string 类型的 handle 和数据的机构来实现。 我们可以使用训练或者验证集的
# output_types和output_shapes 属性来作为数据的结构定义，因为他们都一样
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(
    handle, training_dataset.output_types, training_dataset.output_shapes)
next_element = iterator.get_next()

# You can use feedable iterators with a variety of different kinds of iterator
# (such as one-shot and initializable iterators).
training_iterator = training_dataset.make_one_shot_iterator()
validation_iterator = validation_dataset.make_initializable_iterator()

# 需要注意的一点是，string_handle() 方法返回的是一个 Tensor，只有运行一个 Tensor 才会返回 string 类型的 handle。
# 不然，程序会报错。 
training_handle = sess.run(training_iterator.string_handle())
validation_handle = sess.run(validation_iterator.string_handle())

for _ in range(3):
    print('Epoch {} Training begin'.format(str(_)))
    for _ in range(2):
        print(sess.run(next_element, feed_dict={handle: training_handle}))
    print('Epoch {} Training end'.format(str(_)))
    
    print('Epoch {} Validating begin'.format(str(_)))
    # Run one pass over the validation dataset.
    sess.run(validation_iterator.initializer)
    for _ in range(5):
        print(sess.run(next_element, feed_dict={handle: validation_handle}))
    print('Epoch {} Validating end'.format(str(_)))

Epoch 0 Training begin
-9
-2
Epoch 1 Training end
Epoch 1 Validating begin
0
1
2
3
4
Epoch 4 Validating end
Epoch 1 Training begin
-3
1
Epoch 1 Training end
Epoch 1 Validating begin
0
1
2
3
4
Epoch 4 Validating end
Epoch 2 Training begin
-5
-5
Epoch 1 Training end
Epoch 1 Validating begin
0
1
2
3
4
Epoch 4 Validating end


如果用图表的形式加深理解的话，那就是可馈送 Iterator 的方式，

可以自主决定用哪个 Iterator，就好比不同的水池有不同的水管，不需要用同一根水管接到不同的水池当中去了。

可馈送的 Iterator 和可重新初始化的 Iterator 非常相似，

但是，可馈送的 Iterator 在不同的 Iterator 切换的时候，可以做到不从头开始。